# Notebook 7 - Request WCS Elevation Data from Geobasis NRW - Muenster Basin

[GemGIS](https://github.com/cgre-aachen/gemgis) is a package for geographic information processing for geomodeling. In particular, data is prepared for direct use in [GemPy](https://github.com/cgre-aachen/gempy) via a GemPy Data Class. The package provides functions to process spatial data such as vector data (shape files, geojson files, geopackages), raster data (tiff-files), data retrieved from online services (WMS, WCS, WFS) or KML/XML files. 

At a later stage, functionality will be added to interactively add interfaces and orientations for a model, chosing the extent, defining custom sections and more. In addition, functionality will be provided to export data from GemPy into Geoinformation Systems (=GIS) such as QGIS or ArcGIS and Google Earth. 

# Overview

This notebook presents methods to **automatically download tif-tiles of a 1 m Digital Elevation Model (DEM)** for the State of Northrhine Westfalia, Germany, in particular from the Muenster Basin. Alternatively, the single tiles can be downloaded at https://www.tim-online.nrw.de/tim-online2/ or at https://www.opengeodata.nrw.de/produkte/geobasis/hm/dgm1_xyz/ as single tiles or XYZ files for single communes. In addition, functionality is provided to **merge the tiles to a mosaic and to reduce the resolution of the output raster**. Standard `GemGIS` raster methods can be used to save the raster to disk. 

- [What is a Web Coverage Service?](#WCS)
- [Downloading and Installing GemGIS](#gemgis)
- [Structure of GemGIS](#structure)
- [Importing Libraries](#import)
- [Version Reports](#vreport)

<a id='WCS'></a>
## What is a Web Coverage Service (WCS)?

The Web Coverage Service (WCS) is a standard issued by the Open Geospatial Consortium (OGC). It is designed to simplify remote access to coverages, commonly known as raster maps in GIS. WCS functions over the HTTP protocol, setting out how to obtain data and meta-data using the requests available in that protocol. In practice it allows raster maps to be obtained from a web browser or from any other programme that uses the protocol.

Source: https://www.isric.org/web-coverage-services-wcs


<a id='gemgis'></a>
## Downloading and installing GemGIS

`GemGIS` is under constant development and the latest available version can be downloaded at https://github.com/cgre-aachen/gemgis. A pip version can be found at https://pypi.org/project/gemgis/. A dedicated documentation page will follow.

<a id='structure'></a>
## Structure of GemGIS

The core of `GemGIS` is made of the `GemPyData` class (`gemgis.py`). Its attributes can directly be utilized by `GemPy` making it easier for users to load data. Methods of the `GemPyData` class allow users to directly set these attributes. Multiple other files contain functions to manipulate vector data, raster data, etc.:

* `gemgis.py` - core file containing the `GemPyData` class
* `vector.py` - file containing functions to manipulate vector data
* `raster.py` - file containing functions to manipulate raster data
* `utils.py` - file containing utility functions frequently used for the manipulation of vector/raster data
* `wms.py` - file containing methods to load online services as vector and raster data
* `visualization.py` - file containing functions to simplify plotting of spatial data
* `postprocessing.py` - file containing functions to postprocess GemPy geo_model data


If you have any problems using GemGIS, find a bug or have an idea for a new feature, open an issue at https://github.com/cgre-aachen/gemgis/issues. 

<a id='import'></a>
# Importing Libraries

Apart from creating a GemPyData class later in the tutorial, GemGIS is working with pure GeoDataFrames, Rasterio files and NumPy arrays to provide the user with easy data handling. ***Currently, geopandas version 0.8 is the latest stable version that is supported by GemGIS***. A general introduction to working with rasters and Rasterio objects in GemGIS is provided in the next notebook.

The first step is loading `GemGIS` and the auxiliary libraries `geopandas` and `rasterio` apart from `NumPy` and `Matplotlib`. `GemGIS` will also load `GemPy` the background. If the installation of `GemPy`was not successful, `GemGIS` cannot be used. 

In [ ]:
import sys
import os
sys.path.append('../../../gemgis')
import gemgis as gg
import geopandas as gpd
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
print(gg)

<a id='vreport'></a>
# Version Reports
Reporting your Python environment's package versions and hardware resources by Scooby - https://github.com/banesullivan/scooby. This overview shows the packages and their versions used to run this notebook. Upgrading or downgrading to older or newer versions may break the notebook.

In [ ]:
gg.Report()

# Request Elevation Data from a WCS Server for the Aachen Area

As a WMS Server only provides image data as shown in [Tutorial 3](https://github.com/cgre-aachen/gemgis/blob/master/notebooks/03_GemGIS_Working_with_WMS_Services.ipynb), data must be downloaded from a Web Coverage Service (WCS). For this notebook, we will demonstrate how to download sample files for the Aachen Area to illustrate the functionality. 

## Loading WMS Service for Background Imagery

The WMS Service used in Tutorial 3 will also be used here for Background Imagery. 

### Load WMS Layer and Map

In [ ]:
wms = gg.wms.load('https://services.bgr.de/wms/geologie/gbl/?')

In [ ]:
wms.identification.type

In [ ]:
wms.identification.version

In [ ]:
wms.identification.title

In [ ]:
wms.identification.abstract

In [ ]:
list(wms.contents)

In [ ]:
wms['0'].title

In [ ]:
wms['1'].title

In [ ]:
wms['2'].title

In [ ]:
wms['3'].title

In [ ]:
wms['3'].crsOptions

In [ ]:
wms['3'].styles

In [ ]:
wms['4'].title

In [ ]:
wms_map0 = gg.wms.load_as_array('https://services.bgr.de/wms/geologie/gbl/?',
                             '0', 'default', 'EPSG:25832', [300000, 550000,5650000,5850000], [3000, 2000], 'image/png')

In [ ]:
wms_map0[:,:,3]

In [ ]:
plt.figure(figsize = (12,12))
img = plt.imshow(wms_map0[:,:,0], extent= [300000, 550000,5650000,5850000])
plt.colorbar(img)
plt.grid()
plt.ylabel('Y [m]')
plt.xlabel('X [m]')
plt.text(405000,5758000, 'Münster', size = 18)

In [ ]:
wms_map1 = gg.wms.load_as_array('https://services.bgr.de/wms/geologie/gbl/?',
                             '1', 'default', 'EPSG:25832', [300000, 550000,5650000,5850000], [3000, 2000], 'image/png')

In [ ]:
wms_map1.shape

In [ ]:
plt.figure(figsize = (12,12))
img = plt.imshow(wms_map1[:,:,0], extent= [300000, 550000,5650000,5850000])
plt.colorbar(img)
plt.grid()
plt.ylabel('Y [m]')
plt.xlabel('X [m]')
plt.text(405000,5758000, 'Münster', size = 18)

In [ ]:
wms_map2 = gg.wms.load_as_array('https://services.bgr.de/wms/geologie/gbl/?',
                             '2', 'default', 'EPSG:25832', [300000, 550000,5650000,5850000], [3000, 2000], 'image/png')

In [ ]:
wms_map2[:,:,3]

In [ ]:
wms_map4 = gg.wms.load_as_array('https://services.bgr.de/wms/geologie/gbl/?',
                             '4', 'default', 'EPSG:25832', [300000, 550000,5650000,5850000], [3000, 2000], 'image/png')

In [ ]:
wms_map4[:,:,3]

In [ ]:
wms.getOperationByName('GetMap').formatOptions

In [ ]:
wms_map3 = gg.wms.load_as_array('https://services.bgr.de/wms/geologie/gbl/?',
                             '3', 'default', 'EPSG:25832', [300000, 550000,5650000,5850000], [3000, 2000], 'image/png')

In [ ]:
wms_map3

### Plot WMS Data

In [ ]:
plt.figure(figsize = (12,12))
img = plt.imshow(wms_map3, extent= [300000, 550000,5650000,5850000])
plt.colorbar(img)
plt.grid()
plt.ylabel('Y [m]')
plt.xlabel('X [m]')
plt.text(405000,5758000, 'Münster', size = 18)

In [ ]:
[op.name for op in wms.operations]

In [ ]:
wms.getOperationByName('GetFeatureInfo').methods

In [ ]:
wms.getOperationByName('GetFeatureInfo').formatOptions

In [ ]:
wms.getOperationByName('GetFeatureInfo')

In [ ]:
for i in range(10):
    for j in range(10):
        wms_feature = wms.getfeatureinfo(layers=['0'],  
                       srs='EPSG:25832', 
                       bbox=(300000, 550000,5650000,5850000), 
                       size=(10,10), 
                       #format='image/png',
                       #query_layers=['1'],
                       info_format='application/geojson',
                       #PixelValue for xy
                       xy=(i,j))
        data = json.loads(wms_feature.read())
        print(data)

In [ ]:
import pandas as pd
pd.DataFrame.from_dict(data)

In [ ]:
out = open('../../../getfeatureinfo-response.xml', 'wb')
out.write(wms_feature.read())
out.close()

In [ ]:
import io
import json

json.loads(wms_feature.read())

## Load WCS Layer

The WCS Server is being accessed via OWSLib. The attributes `url` and `version` are needed for the following request.

In [ ]:
wcs_url = 'https://www.wcs.nrw.de/geobasis/wcs_nw_dgm'
wcs = gg.misc.load_wcs(url=wcs_url)
print(type(wcs))
wcs

In [ ]:
wcs.url

In [ ]:
wcs.version

## Creating and Executing WCS Request

The WCS needs to be created by providing min and max values for X and Y locations. Here, a for loop is created to automatically download four tiles with an extent of 2 by 2 km each. Due to their size, they will be saved outside the repository. 

In [ ]:
xmin = 360000
xmax = 500000
ymin = 5700000
ymax = 5800000
size = 2000

x = xmax-xmin
print('Extent X: ', x, ' m')
y = ymax-ymin
print('Extent Y: ', y, ' m')

print('Number of tiles in X directions: ', int(x/size))
print('Number of tiles in Y directions: ', int(y/size))

for i in tqdm(range(int(x/size))):
    for j in range(int(y/size)):
        if not os.path.exists('../../../Tiles_Muenster/tile_%d_%d_%d_%d.tif' % (xmin+i*size, xmin+(i+1)*size,ymin+j*size, ymin+(j+1)*size)):
            url = gg.misc.create_request(wcs.url, wcs.version, 'nw_dgm', 'image/tiff',[xmin+i*size, xmin+(i+1)*size,ymin+j*size, ymin+(j+1)*size], 'test')
            gg.misc.execute_request(url, '../../../Tiles_Muenster/tile_%d_%d_%d_%d.tif' % (xmin+i*size, xmin+(i+1)*size,ymin+j*size, ymin+(j+1)*size))
        else:
            pass

## Create List of File paths

In [ ]:
dem_fps = gg.misc.create_filepaths(dirpath = '../../../Tiles_Muenster/', search_criteria='tile*.tif')
dem_fps[:4]

## Create List of Tiles

The above created list of file paths is automatically being created when executing the function below. In addition, a list of the loaded tiles is created.

In [ ]:
src_files_to_mosaic = gg.misc.create_src_list(dirpath ='../../../Tiles_Muenster/', search_criteria='tile*.tif')
src_files_to_mosaic[:4]

## Merge tiles to mosaic with different resolutions

The single files can now automatically be merged to form a mosaic.

In [ ]:
mosaic, out_trans = gg.misc.merge_tiles(src_files_to_mosaic, res=50)

## Save Rasters to disc

## Plot DEM

The mosaic/DEM can now be plotted using the built-in rasterio functionality or using matplotlib. 

In [ ]:
from rasterio.plot import show
show(mosaic, cmap='terrain', vmax=400)

In [ ]:
plt.figure(figsize = (12,12))
plt.imshow(wms_map, extent= [300000, 550000,5650000,5850000])
plt.grid()
plt.ylabel('Y [m]')
plt.xlabel('X [m]')
plt.text(405000,5758000, 'Münster', size = 18)
im = plt.imshow(mosaic, cmap='terrain', vmax=400, extent = [360000,500000,5700000,5800000])